# Stage-2 Handoff Validation (Orthodox / End-to-End)

このノートは、**Stage-1（ML）→ Stage-2（defer gate / handoff縮退）→ Agent/LLM（必要なら）** のパイプラインを、
**学術・実務で筋が良い（オーソドックスな）** 形で検証します。

評価は大きく2段に分けます：

1. **Gate評価（Stage-2の選別品質）**  
   - Stage-1 の誤り `err = (stage1_pred != y_true)` をどれだけ handoff に回収できたか  
   - `error_capture_recall`, `handoff_precision`  
   - 可能なら **Stage-1 handoff領域内**（`stage1_decision=="handoff_to_agent"`）でも同様に出します

2. **End-to-End評価（最終判定品質）**  
   - Stage-2 handoff に入った domain だけ Agent を実行し、結果が取れたら **Stage-1予測を上書き**  
   - 取れなかった場合は **Stage-1へフォールバック**（運用で筋が良い）  
   - ALL / AUTO（handoff外）/ HANDOFF（handoff内）に分けて混同行列・主要指標を出します

---

## 入力（I/Fは変更しない）
- `artifacts/<RUN_ID>/results/stage1_decisions_latest.csv`
- `artifacts/<RUN_ID>/handoff/handoff_candidates_latest.csv`

## 出力
- Gate指標（ALL / Stage-1 handoff領域）
- Agent単体指標（評価用サンプル）
- End-to-End指標（ALL / AUTO / HANDOFF）
- `artifacts/<RUN_ID>/results/stage2_validation/` 配下に CSV/JSON を保存


In [1]:
# [ChangeLog]
# 2026-01-01: Stage-2 handoff（混在ラベル）を「ゲート評価 + End-to-End評価」に分解したオーソドックス検証フローへ整理
# 2026-01-01: IntCastingNaNError 対策（agent予測が欠損するケースで fillna→astype(int)）
# 2026-01-01: benign追加サンプルはデフォルトOFF（必要なら hard benign をONにしてストレステスト）


In [2]:
# ============================================
# Cell 0: パラメータ（ここだけ変えればOK）
# ============================================
import os

# ▼ Stage-2 handoff（=Agentを回す対象）から何件評価するか
#   - 数値: その件数だけサンプル
#   - "ALL" / 0 / -1: 全件
DEFAULT_N_SAMPLE = "3000"

# ▼ 追加benign（ストレステスト用）: デフォルトは 0（追加しない）
#   - hard benign は「Stage-1がphish寄りと思っている正常」を上位から取る
DEFAULT_N_BENIGN_SAMPLE = "0"
DEFAULT_N_BENIGN_HARD_SAMPLE = "0"   # 例: "100" にするとFP耐性を確認しやすい

DEFAULT_RANDOM_STATE = "42"

# ▼ コスト関数（任意）
#   cost = FN_COST*FN + FP_COST*FP + HANDOFF_COST*handoff_count
DEFAULT_FN_COST = "3.0"
DEFAULT_FP_COST = "1.0"
DEFAULT_HANDOFF_COST = "0.0"

def _normalize_env(name: str, default: str) -> str:
    v = os.getenv(name, default)
    s = str(v).strip()
    if s.upper() == "ALL":
        return "ALL"
    try:
        n = int(float(s))
        return "ALL" if n <= 0 else str(n)
    except Exception:
        # 文字列のまま（後段でエラーにする）
        return s

# env で上書きできる形に統一
os.environ["N_SAMPLE"] = _normalize_env("N_SAMPLE", DEFAULT_N_SAMPLE)
os.environ["N_BENIGN_SAMPLE"] = _normalize_env("N_BENIGN_SAMPLE", DEFAULT_N_BENIGN_SAMPLE)
os.environ["N_BENIGN_HARD_SAMPLE"] = _normalize_env("N_BENIGN_HARD_SAMPLE", DEFAULT_N_BENIGN_HARD_SAMPLE)
os.environ["RANDOM_STATE"] = os.getenv("RANDOM_STATE", DEFAULT_RANDOM_STATE)

os.environ["FN_COST"] = os.getenv("FN_COST", DEFAULT_FN_COST)
os.environ["FP_COST"] = os.getenv("FP_COST", DEFAULT_FP_COST)
os.environ["HANDOFF_COST"] = os.getenv("HANDOFF_COST", DEFAULT_HANDOFF_COST)

print("[SET]",
      "N_SAMPLE=", os.environ["N_SAMPLE"],
      "N_BENIGN_SAMPLE=", os.environ["N_BENIGN_SAMPLE"],
      "N_BENIGN_HARD_SAMPLE=", os.environ["N_BENIGN_HARD_SAMPLE"],
      "RANDOM_STATE=", os.environ["RANDOM_STATE"])
print("[SET]",
      "FN_COST=", os.environ["FN_COST"],
      "FP_COST=", os.environ["FP_COST"],
      "HANDOFF_COST=", os.environ["HANDOFF_COST"])


[SET] N_SAMPLE= 3000 N_BENIGN_SAMPLE= ALL N_BENIGN_HARD_SAMPLE= ALL RANDOM_STATE= 42
[SET] FN_COST= 3.0 FP_COST= 1.0 HANDOFF_COST= 0.0


In [3]:
# ============================================
# Cell 1: 環境設定（RUN_ID / paths / ディレクトリ決定）
#  - 02/03 と同じ bootstrap 方式で RUN_ID を確定
# ============================================
import os, sys, importlib
from pathlib import Path
import datetime

# プロジェクトルート推定（必要なら NEXTSTEP_BASE_DIR を export）
BASE_DIR = Path(os.environ.get("NEXTSTEP_BASE_DIR", ".")).resolve()
if str(BASE_DIR) not in sys.path:
    sys.path.insert(0, str(BASE_DIR))

# RUN_ID の決定（env→run_id.txt→Part3→latest…）
import run_id_registry as runreg
rid = runreg.bootstrap()
os.environ["RUN_ID"] = rid

# _compat.paths（env RUN_ID を反映）
try:
    import _compat.paths as paths
except ImportError:
    import paths as paths  # fallback
importlib.reload(paths)

print("[NX] RUN_ID =", rid, "| paths.RUN_ID =", getattr(paths, "RUN_ID", None))
assert getattr(paths, "RUN_ID", None) == rid, f"RUN_ID mismatch: rid={rid} paths.RUN_ID={getattr(paths,'RUN_ID',None)}"
RUN_ID = rid

# ディレクトリ
ARTIFACTS_DIR = Path(paths.ARTIFACTS) if hasattr(paths, "ARTIFACTS") else (BASE_DIR / "artifacts" / RUN_ID)
RAW_DIR       = Path(paths.compat_base_dirs["raw"])
PROCESSED_DIR = Path(paths.compat_base_dirs["data"])
MODELS_DIR    = Path(paths.compat_base_dirs["models"])
RESULTS_DIR   = Path(paths.compat_base_dirs["results"])
HANDOFF_DIR   = Path(paths.compat_base_dirs["handoff"])
LOGS_DIR      = Path(paths.compat_base_dirs["logs"])
TRACES_DIR    = Path(paths.compat_base_dirs["traces"])

print(f"[INFO] BASE_DIR      = {BASE_DIR}")
print(f"[INFO] RUN_ID        = {RUN_ID}")
print(f"[INFO] ARTIFACTS_DIR = {ARTIFACTS_DIR}")
print(f"[INFO] RESULTS_DIR   = {RESULTS_DIR}")
print(f"[INFO] HANDOFF_DIR   = {HANDOFF_DIR}")
print(f"[INFO] LOGS_DIR      = {LOGS_DIR}")

# 存在チェック（必要ファイル）
print("\n[CHECK]")
print("  stage1_decisions_latest.csv:", (RESULTS_DIR / "stage1_decisions_latest.csv"))
print("  handoff_candidates_latest.csv:", (HANDOFF_DIR / "handoff_candidates_latest.csv"))


[NX] RUN_ID = 2026-01-03_222059 | paths.RUN_ID = 2026-01-03_222059
[INFO] BASE_DIR      = /data/hdd/asomura/nextstep
[INFO] RUN_ID        = 2026-01-03_222059
[INFO] ARTIFACTS_DIR = artifacts/2026-01-03_222059
[INFO] RESULTS_DIR   = artifacts/2026-01-03_222059/results
[INFO] HANDOFF_DIR   = artifacts/2026-01-03_222059/handoff
[INFO] LOGS_DIR      = artifacts/2026-01-03_222059/logs

[CHECK]
  stage1_decisions_latest.csv: artifacts/2026-01-03_222059/results/stage1_decisions_latest.csv
  handoff_candidates_latest.csv: artifacts/2026-01-03_222059/handoff/handoff_candidates_latest.csv


In [4]:
# ============================================
# Cell 2: 指標計算ヘルパー（混同行列 / Fβ / gate指標 / cost）
# ============================================
from dataclasses import dataclass
import numpy as np
import pandas as pd

@dataclass
class ConfusionResult:
    TP: int
    FP: int
    TN: int
    FN: int
    precision: float
    recall: float
    f1: float
    fbeta: float
    fpr: float

def _safe_int_series(x: pd.Series) -> pd.Series:
    # True/False, 0/1, "true"/"false" 等に耐性を持たせる
    if x.dtype == bool:
        return x.astype(int)
    y = pd.to_numeric(x, errors="coerce")
    # "True"/"False" 対応（to_numericでNaNになるので）
    if y.isna().any():
        x2 = x.astype(str).str.strip().str.lower()
        y2 = x2.map({"true": 1, "false": 0})
        y = y.fillna(y2)
    return y.fillna(0).astype(int)

def confusion_from_arrays(y_true, y_pred, *, beta: float = 2.0) -> ConfusionResult:
    y_true = np.asarray(y_true).astype(int)
    y_pred = np.asarray(y_pred).astype(int)

    tp = int(((y_true == 1) & (y_pred == 1)).sum())
    fp = int(((y_true == 0) & (y_pred == 1)).sum())
    tn = int(((y_true == 0) & (y_pred == 0)).sum())
    fn = int(((y_true == 1) & (y_pred == 0)).sum())

    precision = tp / (tp + fp) if (tp + fp) else 0.0
    recall = tp / (tp + fn) if (tp + fn) else 0.0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) else 0.0
    b2 = beta * beta
    fbeta = ((1 + b2) * precision * recall / (b2 * precision + recall)) if (b2 * precision + recall) else 0.0
    fpr = fp / (fp + tn) if (fp + tn) else 0.0

    return ConfusionResult(tp, fp, tn, fn, precision, recall, f1, fbeta, fpr)

def compute_cls_metrics(df: pd.DataFrame, *, label_col="label", pred_col="pred", beta: float = 2.0) -> ConfusionResult:
    y_true = _safe_int_series(df[label_col])
    y_pred = _safe_int_series(df[pred_col])
    return confusion_from_arrays(y_true, y_pred, beta=beta)

def compute_gate_metrics(err: pd.Series, handoff: pd.Series) -> dict:
    # err: 1=Stage-1が誤り / 0=正解
    # handoff: 1=handoff / 0=auto
    e = _safe_int_series(err)
    h = _safe_int_series(handoff)
    tp = int(((e == 1) & (h == 1)).sum())  # captured errors
    fn = int(((e == 1) & (h == 0)).sum())  # missed errors
    fp = int(((e == 0) & (h == 1)).sum())  # unnecessary handoff
    tn = int(((e == 0) & (h == 0)).sum())  # correct auto

    recall = tp / (tp + fn) if (tp + fn) else 0.0
    precision = tp / (tp + fp) if (tp + fp) else 0.0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) else 0.0
    return {
        "TP_captured_errors": tp,
        "FP_unneeded_handoff": fp,
        "FN_missed_errors": fn,
        "TN_correct_auto": tn,
        "error_capture_recall": recall,
        "handoff_precision": precision,
        "f1": f1,
    }

def cost_from_confusion(res: ConfusionResult, *, fn_cost: float, fp_cost: float, handoff_cost: float, handoff_count: int) -> float:
    return fn_cost * res.FN + fp_cost * res.FP + handoff_cost * int(handoff_count)

def _fmt_pct(x: float) -> str:
    return f"{100*x:.3f}%"

print("[OK] helpers ready")


[OK] helpers ready


In [5]:
# ============================================
# Cell 3: データ読込（Stage-1 ALL / Stage-2 handoff） + Gate評価 + target_df作成
# ============================================
from pathlib import Path
import pandas as pd
import numpy as np

# パラメータ
def _parse_n(s: str) -> int:
    s = str(s).strip()
    if s.upper() == "ALL":
        return -1
    return int(float(s))

N_SAMPLE = _parse_n(os.environ.get("N_SAMPLE", "100"))
N_BENIGN_SAMPLE = _parse_n(os.environ.get("N_BENIGN_SAMPLE", "0"))
N_BENIGN_HARD_SAMPLE = _parse_n(os.environ.get("N_BENIGN_HARD_SAMPLE", "0"))
RANDOM_STATE = int(os.environ.get("RANDOM_STATE", "42"))

FN_COST = float(os.environ.get("FN_COST", "3.0"))
FP_COST = float(os.environ.get("FP_COST", "1.0"))
HANDOFF_COST = float(os.environ.get("HANDOFF_COST", "0.0"))

print(f"[INFO] N_SAMPLE={N_SAMPLE}, N_BENIGN_SAMPLE={N_BENIGN_SAMPLE}, N_BENIGN_HARD_SAMPLE={N_BENIGN_HARD_SAMPLE}")
print(f"[INFO] COST: FN_COST={FN_COST}, FP_COST={FP_COST}, HANDOFF_COST={HANDOFF_COST}")

# 入力ファイル
STAGE1_CSV = Path(RESULTS_DIR) / "stage1_decisions_latest.csv"
HANDOFF_CSV = Path(HANDOFF_DIR) / "handoff_candidates_latest.csv"

if not STAGE1_CSV.exists():
    raise FileNotFoundError(f"not found: {STAGE1_CSV}")
if not HANDOFF_CSV.exists():
    raise FileNotFoundError(f"not found: {HANDOFF_CSV}")

full_df = pd.read_csv(STAGE1_CSV)
handoff_all = pd.read_csv(HANDOFF_CSV)

# 正規化
if "y_true" in full_df.columns:
    full_df["label"] = full_df["y_true"].astype(int)
elif "label" in full_df.columns:
    full_df["label"] = full_df["label"].astype(int)
else:
    raise RuntimeError("stage1_decisions_latest.csv must have y_true or label")

full_df["stage1_pred"] = (full_df["ml_probability"] >= 0.5).astype(int)

if "y_true" in handoff_all.columns:
    handoff_all["label"] = handoff_all["y_true"].astype(int)
elif "label" in handoff_all.columns:
    handoff_all["label"] = handoff_all["label"].astype(int)
else:
    raise RuntimeError("handoff_candidates_latest.csv must have y_true or label")

# Stage-2 handoff mask（ALL test）
handoff_domains = set(handoff_all["domain"].astype(str))
full_df["stage2_handoff"] = full_df["domain"].astype(str).isin(handoff_domains).astype(int)
full_df["stage1_err"] = (full_df["stage1_pred"] != full_df["label"]).astype(int)

print("\n[SIZE]")
print("  ALL test:", len(full_df))
print("  Stage-2 handoff(topK):", int(full_df["stage2_handoff"].sum()))

# Stage-1 baseline（ALL）
m_stage1_all = compute_cls_metrics(full_df.assign(pred=full_df["stage1_pred"]), label_col="label", pred_col="pred", beta=2.0)
print("\n[Stage-1 baseline on ALL test] (threshold=0.5)")
print(m_stage1_all)

# Gate評価（Stage-2 handoff as defer gate）
gate_all = compute_gate_metrics(full_df["stage1_err"], full_df["stage2_handoff"])
print("\n[Gate metrics: Stage-2 handoff vs Stage-1 errors] (ALL test)")
print(gate_all)

# Stage-1 handoff領域があれば、その中でのGate評価も出す（02のBudget-Evalと同型）
if "stage1_decision" in full_df.columns:
    region = (full_df["stage1_decision"].astype(str) == "handoff_to_agent")
    if region.any():
        gate_region = compute_gate_metrics(full_df.loc[region, "stage1_err"], full_df.loc[region, "stage2_handoff"])
        print("\n[Gate metrics: within Stage-1 handoff region only]")
        print("  stage1_handoff_region:", int(region.sum()))
        print(gate_region)
    else:
        print("\n[INFO] stage1_decision column exists but no 'handoff_to_agent' rows")
else:
    print("\n[INFO] stage1_decision not found; skip region-gate metrics")

# handoff内のラベル混在（研究上の前提確認）
mix = handoff_all["label"].value_counts().to_dict()
print("\n[Stage-2 handoff label mix]")
print("  label_counts:", mix)

# ---------------------------------------------------------
# target_df（Agent評価対象）を作る
# ---------------------------------------------------------
def _sample(df: pd.DataFrame, n: int) -> pd.DataFrame:
    if n <= 0 or n >= len(df):
        return df.copy()
    return df.sample(n=n, random_state=RANDOM_STATE).copy()

parts = []

# 1) Stage-2 handoff（本線）
s2 = _sample(handoff_all, N_SAMPLE)
s2 = s2.copy()
s2["eval_group"] = "stage2_handoff"
parts.append(s2)

# 2) benign random（任意）
if N_BENIGN_SAMPLE > 0:
    pool = full_df[(full_df["label"] == 0) & (~full_df["domain"].astype(str).isin(handoff_domains))].copy()
    b = _sample(pool, N_BENIGN_SAMPLE)
    b = b.rename(columns={"label": "y_true"}) if "y_true" not in b.columns else b
    b["y_true"] = 0
    b["eval_group"] = "benign_random"
    # 必要最小列に寄せる（列が多いとmergeが重い）
    keep_cols = [c for c in ["domain","ml_probability","y_true","source","stage1_decision"] if c in b.columns]
    parts.append(b[keep_cols])

# 3) benign hard（任意）
if N_BENIGN_HARD_SAMPLE > 0:
    pool = full_df[(full_df["label"] == 0) & (~full_df["domain"].astype(str).isin(handoff_domains))].copy()
    pool = pool.sort_values("ml_probability", ascending=False)
    b = pool.head(N_BENIGN_HARD_SAMPLE).copy()
    b = b.rename(columns={"label": "y_true"}) if "y_true" not in b.columns else b
    b["y_true"] = 0
    b["eval_group"] = "benign_hard"
    keep_cols = [c for c in ["domain","ml_probability","y_true","source","stage1_decision"] if c in b.columns]
    parts.append(b[keep_cols])

target_df = pd.concat(parts, ignore_index=True)
target_df["domain"] = target_df["domain"].astype(str)
target_df = target_df.drop_duplicates(subset=["domain"], keep="first").reset_index(drop=True)

print("\n[TARGET]")
print("  target_df rows:", len(target_df))
print("  groups:", target_df["eval_group"].value_counts().to_dict())
print("  label mix:", target_df["y_true"].astype(int).value_counts().to_dict())

# target_dfでのStage-1 baseline誤り率（ざっくり）
if "y_true" in target_df.columns:
    y = target_df["y_true"].astype(int)
    pred = (target_df["ml_probability"] >= 0.5).astype(int)
    err = (pred != y).mean()
    print(f"\n[Stage-1 baseline error rate on target_df] {err:.4f}")

display(target_df.head(5))

# Stage-2のbudget_evalがあれば表示（整合性チェック用）
s2_eval_json = Path(RESULTS_DIR) / "stage2_budget_eval.json"
if s2_eval_json.exists():
    import json
    j = json.loads(s2_eval_json.read_text(encoding="utf-8"))
    print("\n[INFO] stage2_budget_eval.json found:", s2_eval_json)
    # 目につくキーだけ表示
    keys = ["run_id","handoff_budget","auto_errors","auto_error_rate","error_capture_recall","handoff_precision"]
    show = {k: j.get(k) for k in keys if k in j}
    if show:
        print("  summary:", show)
else:
    print("\n[INFO] stage2_budget_eval.json not found (skip)")


[INFO] N_SAMPLE=3000, N_BENIGN_SAMPLE=-1, N_BENIGN_HARD_SAMPLE=-1
[INFO] COST: FN_COST=3.0, FP_COST=1.0, HANDOFF_COST=0.0

[SIZE]
  ALL test: 128067
  Stage-2 handoff(topK): 7710

[Stage-1 baseline on ALL test] (threshold=0.5)
ConfusionResult(TP=61811, FP=538, TN=63496, FN=2222, precision=0.9913711527049351, recall=0.9652991426295816, f1=0.978161447041509, fbeta=0.9704032579651533, fpr=0.008401786550894836)

[Gate metrics: Stage-2 handoff vs Stage-1 errors] (ALL test)
{'TP_captured_errors': 1717, 'FP_unneeded_handoff': 5993, 'FN_missed_errors': 1043, 'TN_correct_auto': 119314, 'error_capture_recall': 0.6221014492753624, 'handoff_precision': 0.22269779507133594, 'f1': 0.3279847182425979}

[Gate metrics: within Stage-1 handoff region only]
  stage1_handoff_region: 64225
{'TP_captured_errors': 1717, 'FP_unneeded_handoff': 5993, 'FN_missed_errors': 1038, 'TN_correct_auto': 55477, 'error_capture_recall': 0.6232304900181488, 'handoff_precision': 0.22269779507133594, 'f1': 0.32814142379359773

,domain,source,ml_probability,stage1_decision,y_true,label,eval_group
0,scelgocasa.com,jpcert,0.176781,handoff_to_agent,1,1,stage2_handoff
1,dickrobinson.com,trusted,0.125159,handoff_to_agent,0,0,stage2_handoff
2,4qd.co.uk,trusted,0.128810,handoff_to_agent,0,0,stage2_handoff
3,amazon-xq.top,certificates,0.615849,handoff_to_agent,1,1,stage2_handoff
4,iscarpino.com,jpcert,0.174722,handoff_to_agent,1,1,stage2_handoff



[INFO] stage2_budget_eval.json found: artifacts/2026-01-03_222059/results/stage2_budget_eval.json
  summary: {'auto_errors': 1043, 'auto_error_rate': 0.008665885656837576}


In [6]:
# ============================================
# Cell 4: external_data のロード（可能なら）
#  - 98-output の前提ファイルがあれば利用
#  - 無ければ最小限で続行（agentが動かない場合はここで気づける）
# ============================================
import pickle
from pathlib import Path

external_data = {}
handoff_path = Path(HANDOFF_DIR) / "04-3_llm_tools_setup_with_tools.pkl"

if handoff_path.exists():
    with open(handoff_path, "rb") as f:
        obj = pickle.load(f)
    if isinstance(obj, dict) and "external_data" in obj and isinstance(obj["external_data"], dict):
        external_data = obj["external_data"]
        print("[OK] loaded external_data from", handoff_path.name, "keys=", len(external_data))
    elif isinstance(obj, dict):
        # そのまま external_data として扱う（互換）
        external_data = obj
        print("[WARN] pickle is dict but has no 'external_data'; using whole dict as external_data, keys=", len(external_data))
    else:
        print("[WARN] unexpected pickle type:", type(obj), " -> external_data empty")
else:
    print("[WARN] not found:", handoff_path)
    print("       -> external_data will be empty (agent may still run depending on defaults).")

# brand_keywords が無いと brand tool が弱くなるので最低限
external_data.setdefault("brand_keywords", [])
print("[INFO] brand_keywords:", len(external_data.get("brand_keywords", [])))


[WARN] pickle is dict but has no 'external_data'; using whole dict as external_data, keys= 10
[INFO] brand_keywords: 100


In [7]:
# ============================================
# Optional Cell: TLDリストの動的生成（DB分析）
#  - heavyなのでデフォルトOFF
# ============================================
ENABLE_DB_TLD_REFRESH = False  # True にすると実行します

if not ENABLE_DB_TLD_REFRESH:
    print("[SKIP] ENABLE_DB_TLD_REFRESH=False")
else:
    try:
        import psycopg2
        from psycopg2.extras import RealDictCursor
        from collections import Counter
        from urllib.parse import urlparse
        import random

        print("🔧 Generating TLD data from Database Analysis...")

        # config.json から接続先を読む（環境に合わせて）
        import json
        cfg_path = Path(os.environ.get("CONFIG_JSON", str(BASE_DIR / "config.json")))
        if not cfg_path.exists():
            cfg_path = Path(BASE_DIR / "_compat" / "config.json")
        cfg = json.loads(cfg_path.read_text(encoding="utf-8"))
        db_cfg = cfg.get("db", cfg.get("database", {}))
        conn = psycopg2.connect(**db_cfg)

        def extract_tld(domain):
            if not domain or '.' not in domain:
                return None
            return domain.split('.')[-1].lower()

        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute("SELECT domain FROM certificates WHERE status='SUCCESS' AND domain IS NOT NULL LIMIT 200000;")
            phish_domains = [r["domain"] for r in cur.fetchall()]
            cur.execute("SELECT domain FROM trusted_certificates WHERE status='SUCCESS' AND domain IS NOT NULL LIMIT 200000;")
            trusted_domains = [r["domain"] for r in cur.fetchall()]

        conn.close()

        random.seed(42)
        phish_unique = list(set(phish_domains))
        trusted_unique = list(set(trusted_domains))
        m = min(len(phish_unique), len(trusted_unique))
        phish_bal = random.sample(phish_unique, m)
        trusted_bal = random.sample(trusted_unique, m)

        phish_tlds = Counter([extract_tld(d) for d in phish_bal if extract_tld(d)])
        trusted_tlds = Counter([extract_tld(d) for d in trusted_bal if extract_tld(d)])

        # 簡易: phish比が高いTLDを risky として採用
        dangerous = []
        for tld, pc in phish_tlds.items():
            tc = trusted_tlds.get(tld, 0)
            if pc >= 50:
                ratio = pc / (tc + 1)
                if ratio >= 5:
                    dangerous.append(tld)
        dangerous = sorted(set(dangerous))
        external_data["dangerous_tlds"] = dangerous
        print("[OK] dangerous_tlds updated:", len(dangerous))
    except Exception as e:
        print("[WARN] DB TLD refresh failed:", type(e).__name__, e)


[SKIP] ENABLE_DB_TLD_REFRESH=False


In [8]:
# ============================================
# Cell 5: Agent 初期化（Phase6 wiring + LangGraphPhishingAgent）
# ============================================
import os, sys
from pathlib import Path

# config.json の場所（環境に合わせて）
cfg_path = Path(os.environ.get("CONFIG_JSON", str(BASE_DIR / "config.json"))).resolve()
if not cfg_path.exists():
    alt = Path(BASE_DIR / "_compat" / "config.json")
    if alt.exists():
        cfg_path = alt.resolve()

os.environ["CONFIG_JSON"] = str(cfg_path)
print("[INFO] CONFIG_JSON =", os.environ["CONFIG_JSON"])

# Phase6 wiring
from phishing_agent.phase6_wiring import wire_phase6
wire_phase6(prefer_compat=True, fake_llm=False)
print("[OK] Phase6 wired")

# Agent
from phishing_agent.langgraph_module import LangGraphPhishingAgent
agent = LangGraphPhishingAgent(
    strict_mode=True,
    use_llm_selection=True,
    use_llm_decision=True,
    config_path=str(cfg_path),
    external_data=external_data,
)
print("[OK] agent initialized:", type(agent))


[INFO] CONFIG_JSON = /data/hdd/asomura/nextstep/config.json


/home/asomura/waseda/phish-core/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[OK] Phase6 wired
[OK] agent initialized: <class 'phishing_agent.langgraph_module.LangGraphPhishingAgent'>


In [9]:
# ============================================
# Cell 5: サンプル評価実行（件数=len(target_df), 超詳細デバッグログ版）
# ============================================

import time
import pandas as pd
from datetime import datetime
import io, sys, json, os
from contextlib import redirect_stdout, redirect_stderr

print(f"[INFO] Starting FULL AGENT evaluation of {len(target_df)} domains (N_SAMPLE={N_SAMPLE})...")
print("=" * 80)

# LLM設定の確認
if 'agent' in globals() and hasattr(agent, 'llm_config'):
    llm_config = agent.llm_config
    if getattr(llm_config, "enabled", False):
        print(f"[INFO] LLM initialized: {llm_config.model}")
    else:
        print("[WARNING] LLM is disabled - results may be limited")
else:
    print("[WARNING] Agent not properly initialized")

# --- Code fingerprint (reproducibility / verification) ------------------------
# [ChangeLog] 2025-12-17: Add code fingerprint (module __file__ + sha256) to each output row.
import hashlib
import importlib
from pathlib import Path
import sys
import datetime as _dt

def _sha256_of_file(path: str, chunk_size: int = 1024 * 1024) -> str:
    """Return sha256 hex digest for a local file. Never raises."""
    try:
        h = hashlib.sha256()
        with open(path, "rb") as f:
            for chunk in iter(lambda: f.read(chunk_size), b""):
                h.update(chunk)
        return h.hexdigest()
    except Exception as e:
        return f"ERROR:{type(e).__name__}"

def _module_file_and_sha(modname: str):
    """Import module and return (file_path, sha256, module_obj)."""
    try:
        mod = importlib.import_module(modname)
        f = getattr(mod, "__file__", None)
        if f:
            f = str(Path(f).resolve())
            sha = _sha256_of_file(f)
        else:
            sha = None
        return f, sha, mod
    except Exception as e:
        return None, f"IMPORT_ERROR:{type(e).__name__}", None

def make_code_fingerprint() -> dict:
    fp = {}
    fp["eval_id"] = _dt.datetime.now().strftime("%Y-%m-%d_%H%M%S")

    # Core modules used by final decision path
    p6_file, p6_sha, _p6_mod = _module_file_and_sha("phishing_agent.phase6_wiring")
    ld_file, ld_sha, ld_mod = _module_file_and_sha("phishing_agent.llm_final_decision")
    lg_file, lg_sha, _lg_mod = _module_file_and_sha("phishing_agent.langgraph_module")

    fp["phase6_wiring_file"] = p6_file
    fp["phase6_wiring_sha256"] = p6_sha
    fp["llm_final_decision_file"] = ld_file
    fp["llm_final_decision_sha256"] = ld_sha
    fp["langgraph_module_file"] = lg_file
    fp["langgraph_module_sha256"] = lg_sha

    fp["phase6_policy_version_code"] = getattr(ld_mod, "PHASE6_POLICY_VERSION", None) if ld_mod else None
    fp["python"] = sys.version.split()[0]
    fp["dual_import_langgraph_module"] = ("langgraph_module" in sys.modules) and ("phishing_agent.langgraph_module" in sys.modules)

    return fp

# Compute once per notebook run (avoid per-domain overhead)
if "CODE_FINGERPRINT" not in globals():
    CODE_FINGERPRINT = make_code_fingerprint()
    CODE_FP_ROW = {
        "eval_id": CODE_FINGERPRINT.get("eval_id"),
        "phase6_policy_version_code": CODE_FINGERPRINT.get("phase6_policy_version_code"),
        "phase6_wiring_file": CODE_FINGERPRINT.get("phase6_wiring_file"),
        "phase6_wiring_sha256": CODE_FINGERPRINT.get("phase6_wiring_sha256"),
        "llm_final_decision_file": CODE_FINGERPRINT.get("llm_final_decision_file"),
        "llm_final_decision_sha256": CODE_FINGERPRINT.get("llm_final_decision_sha256"),
        "langgraph_module_file": CODE_FINGERPRINT.get("langgraph_module_file"),
        "langgraph_module_sha256": CODE_FINGERPRINT.get("langgraph_module_sha256"),
        "dual_import_langgraph_module": CODE_FINGERPRINT.get("dual_import_langgraph_module"),
    }
    print("[INFO] code_fingerprint:", CODE_FP_ROW)
else:
    # Reuse existing in case notebook cells are re-run
    CODE_FP_ROW = globals().get("CODE_FP_ROW", {})

results = []
start_time = time.time()
# --- 保存設定: 500件ごとに追記（分割ファイルなし） -----------------
SAVE_EVERY_ROWS = 500  # 要件: 500件ごとに保存（追記）
if "LOGS_DIR" not in globals() or not globals().get("LOGS_DIR"):
    raise RuntimeError("LOGS_DIR is not set. Run Cell 1 to initialize run_id/paths first.")

OUT_DIR = Path(globals()["LOGS_DIR"])
OUT_DIR.mkdir(parents=True, exist_ok=True)

# eval_id は Notebook実行単位で固定（CODE_FP_ROW があればそれを使う）
_eval_id = None
if "CODE_FP_ROW" in globals() and isinstance(globals().get("CODE_FP_ROW"), dict):
    _eval_id = globals()["CODE_FP_ROW"].get("eval_id")
if not _eval_id:
    _eval_id = datetime.now().strftime("%Y-%m-%d_%H%M%S")

_ts = datetime.now().strftime("%Y-%m-%d_%H%M%S")

# 追記先（run_id/logs 配下へ）
full_eval_path = OUT_DIR / f"stage2_handoff{len(target_df)}_full_eval__evalid_{_eval_id}__ts_{_ts}.csv"
print(f"[INFO] Results will be appended to: {full_eval_path} (flush every {SAVE_EVERY_ROWS} rows)")

_chunk_buf = []
_written_header = False
_written_rows = 0

def _flush_chunk_if_needed(force: bool = False):
    """_chunk_buf を full_eval_path に追記保存（ヘッダは最初だけ）。"""
    global _written_header, _written_rows
    if (not force) and (len(_chunk_buf) < SAVE_EVERY_ROWS):
        return
    if not _chunk_buf:
        return
    df_chunk = pd.DataFrame(_chunk_buf)
    df_chunk.to_csv(full_eval_path, mode="a", header=(not _written_header), index=False)
    _written_header = True
    _written_rows += int(len(_chunk_buf))
    _chunk_buf.clear()

# --- Notebook画面出力の制御（debug_log の情報量は維持） -----------------
# [ChangeLog] 2025-12-15: 大量件数でもJupyterの出力が重くならないよう、画面出力を間引く
TOTAL_N = len(target_df)

# デフォルト: 少数件は従来どおり詳細表示 / 多数件はコンパクト表示
SHOW_DOMAIN_LOG_ON_SCREEN = (TOTAL_N <= 20)

# 環境変数で上書き可: 1/true/yes/on で詳細表示
_env = os.getenv("SHOW_DOMAIN_LOG_ON_SCREEN")
if _env is not None:
    SHOW_DOMAIN_LOG_ON_SCREEN = str(_env).strip().lower() in ("1", "true", "yes", "y", "on")

# 進捗表示の更新頻度（画面を軽くするため、件数に応じて自動調整）
if TOTAL_N <= 30:
    PROGRESS_EVERY = 1
elif TOTAL_N <= 300:
    PROGRESS_EVERY = 10
elif TOTAL_N <= 3000:
    PROGRESS_EVERY = 25
else:
    PROGRESS_EVERY = 50

# Jupyterなら clear_output で表示を置き換え（ログはCSV側に残る）
_USE_CLEAR_OUTPUT = (not SHOW_DOMAIN_LOG_ON_SCREEN)
try:
    from IPython.display import clear_output  # type: ignore
    _HAS_CLEAR_OUTPUT = True
except Exception:
    _HAS_CLEAR_OUTPUT = False

print(f"[INFO] Console output mode: {'VERBOSE' if SHOW_DOMAIN_LOG_ON_SCREEN else 'COMPACT'} "
      f"(PROGRESS_EVERY={PROGRESS_EVERY}, clear_output={_USE_CLEAR_OUTPUT and _HAS_CLEAR_OUTPUT})")

# 進捗カウンタ（画面表示用）
_screen_counts = {"phish": 0, "benign": 0, "error": 0}

# --- 表示用ヘルパー ----------------------------------------------------
def _fmt_issues(issues):
    return ", ".join(issues) if issues else "None"

def _safe_dict(d):
    return d if isinstance(d, dict) else {}

def _log_tool_selection(graph_state, ml_probability):
    gs = _safe_dict(graph_state)
    pre = _safe_dict(gs.get("precheck_hints"))
    ml_cat   = pre.get("ml_category", "-")
    tld_cat  = pre.get("tld_category", "-")
    quick_risk = pre.get("quick_risk", "-")
    known_info = _safe_dict(pre.get("known_domain_info"))
    known_label = known_info.get("label") or known_info.get("kind") or known_info.get("brand")

    selected_tools = gs.get("selected_tools", [])
    flags = _safe_dict(gs.get("tool_execution_flags"))
    llm_used = gs.get("llm_used_selection")
    llm_err  = gs.get("llm_selection_error")

    print("  [ToolSelection]")
    print(f"    ml_probability      : {ml_probability:.3f} (category={ml_cat})")
    print(f"    tld_category        : {tld_cat}  quick_risk={quick_risk}")
    if known_label:
        print(f"    known_domain_info   : {known_label}")
    print(f"    selected_tools      : {selected_tools}")
    print(f"    tool_execution_flags: {flags}")
    print(f"    llm_used_selection  : {llm_used}")
    if llm_err:
        print(f"    llm_selection_error : {llm_err}")

    if llm_used:
        strategy = "llm_structured_output"
    else:
        strategy = "ml_bucket_fallback"
    print(f"    selection_strategy  : {strategy}")

    # フォールバック時は、どのバケットポリシーになったかも表示
    if not llm_used:
        if ml_probability < 0.2:
            policy = "ML<0.2 → brand+cert+domain (3 tools)"
        elif ml_probability < 0.5:
            policy = "0.2≦ML<0.5 → brand+cert+domain (3 tools)"
        else:
            policy = "ML≧0.5 → brand+cert (2 tools)"
        print(f"    selection_policy    : {policy}")

def _log_brand_tool(brand_res):
    br = _safe_dict(brand_res)
    if not br:
        print("  [BrandTool] not executed")
        return

    details = _safe_dict(br.get("details"))
    issues  = br.get("detected_issues") or []
    # ※ brand_impersonation_check の構造に合わせて details 側を優先
    brands  = details.get("detected_brands") or br.get("detected_brands") or []
    used_llm    = details.get("used_llm")
    llm_conf    = details.get("llm_confidence")
    llm_reason  = details.get("llm_reasoning")
    #llm_reason  = (details.get("llm_reasoning") or "")[:160]

    print("  [BrandTool]")
    if br.get("_fallback"):
        print("    ⚠ fallback           : True (brand tool exception or disabled)")
    print(f"    risk_score           : {br.get('risk_score')}")
    print(f"    detected_issues      : {issues}")
    print(f"    detected_brands      : {brands}")
    print(f"    used_llm             : {used_llm}")
    print(f"    llm_confidence       : {llm_conf}")
    print(f"    llm_reasoning (head) : {llm_reason}")
    if used_llm:
        print("    ✅ LLM が実行されました (used_llm=True)")
    if brands:
        print(f"    ✅ ブランド検出があります: {brands}")

def _log_cert_tool(cert_res):
    cr = _safe_dict(cert_res)
    if not cr:
        print("  [CertTool] not executed")
        return
    details = _safe_dict(cr.get("details"))
    print("  [CertTool]")
    if cr.get("_fallback"):
        print("    ⚠ fallback      : True (cert tool exception or disabled)")
    print(f"    risk_score      : {cr.get('risk_score')}")
    print(f"    detected_issues : {cr.get('detected_issues') or []}")
    print(f"    issuer          : {details.get('issuer')}")
    print(f"    is_free_ca      : {details.get('is_free_ca')}")
    print(f"    has_org         : {details.get('has_org')}")
    print(f"    valid_days      : {details.get('valid_days')}")
    print(f"    is_short_term   : {details.get('is_short_term')}")
    print(f"    san_count       : {details.get('san_count')}")
    print(f"    is_self_signed  : {details.get('is_self_signed')}")
    print(f"    is_wildcard     : {details.get('is_wildcard')}")

def _log_domain_tool(domain_res):
    dr = _safe_dict(domain_res)
    if not dr:
        print("  [DomainTool] not executed")
        return
    details = _safe_dict(dr.get("details"))
    print("  [DomainTool]")
    if dr.get("_fallback"):
        print("    ⚠ fallback            : True (domain tool exception or disabled)")
    print(f"    risk_score            : {dr.get('risk_score')}")
    print(f"    detected_issues       : {dr.get('detected_issues') or []}")
    print(f"    base_domain           : {details.get('base_domain')}")
    print(f"    domain_length         : {details.get('domain_length')}"
          f" (category={details.get('domain_length_category')})")
    print(f"    tld / tld_category    : {details.get('tld')} / {details.get('tld_category')}")
    print(f"    entropy               : {details.get('entropy')}")
    print(f"    combo_flags           : {details.get('combo_flags')}")
    legit = _safe_dict(details.get('legitimate_check'))
    if legit:
        print(f"    legitimate_check      : is_legit={legit.get('is_legitimate')}"
              f", brand={legit.get('brand')}, conf={legit.get('confidence')}")

def _log_contextual_tool(ctx_res):
    cx = _safe_dict(ctx_res)
    if not cx:
        print("  [Contextual] not executed")
        return
    details = _safe_dict(cx.get("details"))
    print("  [Contextual]")
    if cx.get("_fallback"):
        print("    ⚠ fallback            : True (contextual tool exception or disabled)")
    print(f"    risk_score            : {cx.get('risk_score')}")
    print(f"    detected_issues       : {cx.get('detected_issues') or []}")
    print(f"    ml_probability        : {details.get('ml_probability')}"
          f" (category={details.get('ml_category')})")
    print(f"    tool_average_risk     : {details.get('tool_average_risk')}")
    print(f"    high_risk_hits        : {details.get('high_risk_hits')}")
    known = _safe_dict(details.get('known_domain'))
    if known:
        print(f"    known_domain          : {known}")
    paradox = _safe_dict(details.get('paradox'))
    if paradox:
        print(f"    paradox_info          : {paradox}")

def _log_final_decision(graph_state, result):
    gs = _safe_dict(graph_state)
    dbg = _safe_dict(gs.get("debug_llm_final"))
    dt_list = gs.get("decision_trace") or []
    print("  [FinalDecision]")
    print(f"    use_llm_decision      : {dbg.get('use_llm_decision')}")
    print(f"    llm_is_none           : {dbg.get('llm_is_none')}")
    print(f"    path                  : {dbg.get('path')}")
    print(f"    success               : {dbg.get('success')}")
    if dbg.get("error"):
        print(f"    error                 : {dbg.get('error')}")

    if dt_list:
        last = _safe_dict(dt_list[-1])
        print(f"    phase6_policy_version : {gs.get('phase6_policy_version', last.get('phase6_version'))}")
        print(f"    ctx_risk_score        : {last.get('ctx_score')}")
        rules = [s.get("rule") for s in last.get("policy_trace", []) if isinstance(s, dict) and "rule" in s]
        notes = [s.get("note") for s in last.get("policy_trace", []) if isinstance(s, dict) and "note" in s]
        if rules:
            print(f"    policy_rules          : {rules}")
        if notes:
            print(f"    policy_notes          : {notes}")

    print(f"    ai_is_phishing        : {result.get('ai_is_phishing')} "
          f"(risk_level={result.get('ai_risk_level')}, conf={result.get('ai_confidence'):.2f})")
    print(f"    final_reasoning(head) : {(result.get('reasoning') or '')[:160]}")

def _log_graph_messages(graph_state, prefix="    "):
    """LangGraph のメッセージストリームから、重要そうなものだけ抜粋表示"""
    gs = _safe_dict(graph_state)
    msgs = gs.get("messages") or []
    if not msgs:
        return
    print("  [GraphMessages] (tool_selection / tool_execution / final_decision only)")
    for m in msgs:
        if isinstance(m, dict):
            content = m.get("content", "")
            role = m.get("role", "msg")
        else:
            content = getattr(m, "content", "")
            role = getattr(m, "role", "msg")
        if not isinstance(content, str):
            content = str(content)
        if any(tag in content for tag in ["[tool_selection]", "[tool_execution]", "[final_decision]"]):
            print(f"{prefix}[{role}] {content}")


# --- ログをCSVに残すための標準出力/標準エラーキャプチャ（print/logging両対応） ---
class _TeeIO:
    def __init__(self, primary, buffer, *, enable_primary: bool = True):
        self.primary = primary
        self.buffer = buffer
        self.enable_primary = bool(enable_primary)
    def write(self, s):
        if self.enable_primary:
            self.primary.write(s)
        self.buffer.write(s)
        return len(s)
    def flush(self):
        if self.enable_primary:
            try:
                self.primary.flush()
            except Exception:
                pass
        try:
            self.buffer.flush()
        except Exception:
            pass

def _json_dumps(obj):
    """CSVに安全に載せるためのJSON化（失敗しても落とさない）"""
    try:
        return json.dumps(obj, ensure_ascii=False, default=str)
    except Exception:
        return json.dumps(str(obj), ensure_ascii=False, default=str)

# --- 各ドメインを評価 ---------------------------------------------------
for idx, row in target_df.iterrows():
    before_len = len(results)  # 追記保存用（このイテレーションで追加された行だけ拾う）
    domain = row['domain']
    ml_prob = row['ml_probability']
    _buf = io.StringIO()
    _tee_out = _TeeIO(sys.stdout, _buf, enable_primary=SHOW_DOMAIN_LOG_ON_SCREEN)
    _tee_err = _TeeIO(sys.stderr, _buf, enable_primary=SHOW_DOMAIN_LOG_ON_SCREEN)
    with redirect_stdout(_tee_out), redirect_stderr(_tee_err):

        try:
            eval_start = time.time()
            result = agent.evaluate(domain, ml_prob)
            elapsed = time.time() - eval_start

            # --- 基本情報 ---
            is_phishing = result.get('ai_is_phishing', False)
            confidence  = result.get('ai_confidence', 0.0)
            risk_level  = result.get('ai_risk_level', 'unknown')

            graph_state = _safe_dict(result.get("graph_state"))
            tool_res = _safe_dict(graph_state.get("tool_results")) or _safe_dict(result.get("tool_results"))

            # Phase4 仕様: brand/cert/domain/contextual_risk_assessment は data 本体が入っている想定
            brand_res  = _safe_dict(tool_res.get('brand'))
            cert_res   = _safe_dict(tool_res.get('cert'))
            domain_res = _safe_dict(tool_res.get('domain'))
            ctx_res    = _safe_dict(tool_res.get('contextual_risk_assessment') or tool_res.get('contextual'))

            # Brand details
            brand_details   = _safe_dict(brand_res.get('details'))
            detected_brands = brand_details.get('detected_brands', [])

            # brand_detected / brand_suspected を tool 側のフラグから取得（brands配列の有無で決めない）
            _b_issues = brand_res.get('detected_issues', []) if isinstance(brand_res, dict) else []
            brand_detected_flag = bool(brand_details.get('brand_detected')) or ('brand_detected' in (_b_issues or []))
            brand_suspected_flag = bool(brand_details.get('brand_suspected')) or ('brand_suspected' in (_b_issues or [])) or ('brand_llm_candidate' in (_b_issues or []))


            # Cert details
            cert_details = _safe_dict(cert_res.get('details'))
            cert_issuer  = cert_details.get('issuer', 'unknown')

            # Domain details
            domain_details = _safe_dict(domain_res.get('details'))
            tld_cat        = domain_details.get('tld_category', '-')

            # Contextual issues
            ctx_issues = ctx_res.get('detected_issues', []) if ctx_res else []

            mark = "🔴" if is_phishing else "🟢"
            print("-" * 80)
            print(f"[{idx+1:3}/{len(target_df)}] {mark} {domain:<35} (ML: {ml_prob:.3f} / Time: {elapsed:.2f}s)")
            print(f"🔍 Domain: {domain} (ml_probability={ml_prob:.3f})")

            # --- Tool Selection 詳細 ---
            _log_tool_selection(graph_state, ml_prob)

            # --- 各ツール詳細 ---
            _log_brand_tool(brand_res)
            _log_cert_tool(cert_res)
            _log_domain_tool(domain_res)
            if ctx_res:
                _log_contextual_tool(ctx_res)

            # --- Final Decision 詳細 ---
            _log_final_decision(graph_state, result)

            # --- LangGraph メッセージ抜粋 ---
            _log_graph_messages(graph_state)

            # --- 保存用データの構築 ---
            dbg_final = _safe_dict(graph_state.get("debug_llm_final"))
            results.append({
                **CODE_FP_ROW,
                'domain': domain,
                'ml_probability': ml_prob,
                'ai_is_phishing': is_phishing,
                'ai_confidence': confidence,
                'ai_risk_level': risk_level,
                'processing_time': elapsed,
                'debug_log': _buf.getvalue(),
                'graph_state_json': _json_dumps(graph_state),
                'tool_results_json': _json_dumps(tool_res),
                # Brand
                'brand_detected': brand_detected_flag,
                'brand_suspected': brand_suspected_flag,
                'brands': detected_brands,
                'brand_used_llm': brand_details.get('used_llm'),
                'brand_llm_quality': brand_details.get('llm_candidate_quality'),
                'brand_llm_evidence_token': brand_details.get('llm_evidence_token'),
                'brand_llm_detected_brand': brand_details.get('llm_detected_brand'),
                'brand_llm_confidence': brand_details.get('llm_confidence'),
                'brand_risk_score': brand_res.get('risk_score'),
                'brand_issues': brand_res.get('detected_issues', []),
                # Cert
                'cert_issues': cert_res.get('detected_issues', []),
                'cert_issuer': cert_issuer,
                'cert_score': cert_res.get('risk_score', 0.0),
                # Domain
                'domain_issues': domain_res.get('detected_issues', []),
                'domain_score': domain_res.get('risk_score', 0.0),
                'tld_category': tld_cat,
                # Contextual
                'ctx_issues': ctx_issues,
                'ctx_score': ctx_res.get('risk_score', None) if ctx_res else None,
                # Tool Selection / Final LLM debug
                'tool_selection_llm_used': graph_state.get('llm_used_selection'),
                'tool_selection_llm_error': graph_state.get('llm_selection_error'),
                'final_llm_path': dbg_final.get('path'),
                'final_llm_success': dbg_final.get('success'),
                'phase6_policy_version': graph_state.get('phase6_policy_version'),
                'module_version': result.get('module_version'),
                'error': None,
            })

        except Exception as e:
            elapsed = time.time() - eval_start
            print(f"[{idx+1:3}/{len(target_df)}] ❌ ERROR: {domain} - {str(e)}")
            results.append({
                **CODE_FP_ROW,
                'domain': domain,
                'ml_probability': ml_prob,
                'ai_is_phishing': False,
                'ai_confidence': 0.0,
                'ai_risk_level': 'error',
                'processing_time': elapsed,
                'debug_log': _buf.getvalue(),
                'graph_state_json': _json_dumps({}),
                'tool_results_json': _json_dumps({}),
                'error': str(e),
            })

    # --- 追記保存（500件ごと）: このドメインの結果（results[-1]）をバッファに積み、必要なら追記 ---
    try:
        if len(results) > before_len:
            _chunk_buf.append(results[-1])
            _flush_chunk_if_needed(force=False)
    except Exception:
        # 追記保存の失敗で評価自体を止めない（ログは結果側に残す）
        pass
    # --- 画面表示（コンパクト進捗） ---------------------------------------
    # NOTE: 詳細ログは _buf に全量保存しているので、画面側は軽くする
    try:
        _last = results[-1] if results else None
    except Exception:
        _last = None

    if _last is not None:
        if _last.get("error"):
            _screen_counts["error"] += 1
        else:
            if bool(_last.get("ai_is_phishing")):
                _screen_counts["phish"] += 1
            else:
                _screen_counts["benign"] += 1

        # 詳細ログを画面に出さないモードのときだけ、進捗を間引いて表示
        if (not SHOW_DOMAIN_LOG_ON_SCREEN) and (
            ((idx + 1) % PROGRESS_EVERY == 0) or (idx == (len(target_df) - 1)) or _last.get("error")
        ):
            if _HAS_CLEAR_OUTPUT and _USE_CLEAR_OUTPUT:
                try:
                    clear_output(wait=True)
                except Exception:
                    pass

            elapsed_total = time.time() - start_time
            mark = "❌" if _last.get("error") else ("🔴" if _last.get("ai_is_phishing") else "🟢")
            dom  = _last.get("domain", "-")
            mlp  = float(_last.get("ml_probability") or 0.0)
            conf = float(_last.get("ai_confidence") or 0.0)
            rl   = _last.get("ai_risk_level", "-")
            tsec = float(_last.get("processing_time") or 0.0)

            print(f"[PROGRESS] {idx+1}/{len(target_df)}  phishing={_screen_counts['phish']}  benign={_screen_counts['benign']}  error={_screen_counts['error']}  elapsed={elapsed_total:.1f}s")
            print(f"          last: {mark} {dom} (ML={mlp:.3f} risk={rl} conf={conf:.2f} t={tsec:.2f}s)")
            if _last.get("error"):
                print(f"          last_error: {_last.get('error')}")

# --- 完了処理 ------------------------------------------------------------
total_time = time.time() - start_time
print("=" * 80)
print(f"[INFO] Full evaluation complete! Total time: {total_time:.2f}s")

# DataFrame化（後続セルの分析用にメモリ上にも保持）
results_df = pd.DataFrame(results)

# --- 追記保存: 端数（<500件）も必ず保存して終了 ---
_flush_chunk_if_needed(force=True)

print(f"[INFO] Results saved (append) to: {full_eval_path}  rows_written={_written_rows}")


[PROGRESS] 3000/3000  phishing=777  benign=2223  error=0  elapsed=20560.5s
          last: 🟢 fairlabor.org (ML=0.258 risk=low conf=0.85 t=4.45s)
[INFO] Full evaluation complete! Total time: 20560.50s
[INFO] Results saved (append) to: artifacts/2026-01-03_222059/logs/stage2_handoff3000_full_eval__evalid_2026-01-04_012533__ts_2026-01-04_012533.csv  rows_written=3000


In [10]:
# ============================================
# Cell 7: Agent結果を target_df に結合して評価用 DataFrame を作る
# ============================================
import pandas as pd
import numpy as np

if "results_df" not in globals():
    raise RuntimeError("results_df not found. Run the agent evaluation cell first.")

# agent outputs
agent_cols = [c for c in results_df.columns if c in [
    "domain","ai_is_phishing","ai_confidence","ai_risk_level","processing_time","error"
]]
agent_pred = results_df[agent_cols].copy()
agent_pred["domain"] = agent_pred["domain"].astype(str)

# merge
eval_df = target_df.copy()
eval_df["domain"] = eval_df["domain"].astype(str)
eval_df = eval_df.merge(agent_pred, on="domain", how="left")

# labels/preds
eval_df["label"] = eval_df["y_true"].astype(int)
eval_df["stage1_pred"] = (eval_df["ml_probability"] >= 0.5).astype(int)

# agent_pred (numeric, may be NaN)
eval_df["agent_pred"] = pd.to_numeric(eval_df["ai_is_phishing"], errors="coerce")
# "True"/"False" fallback
if eval_df["agent_pred"].isna().any():
    x = eval_df["ai_is_phishing"].astype(str).str.strip().str.lower()
    eval_df["agent_pred"] = eval_df["agent_pred"].fillna(x.map({"true": 1, "false": 0}))
eval_df["agent_covered"] = eval_df["agent_pred"].notna().astype(int)

# final_pred (for eval subset): agent if available else stage1
eval_df["final_pred"] = eval_df["agent_pred"].fillna(eval_df["stage1_pred"]).astype(int)

print("[INFO] eval_df rows:", len(eval_df))
print("[INFO] agent_covered:", int(eval_df["agent_covered"].sum()), "/", len(eval_df))

display(eval_df.head(5))

# --- 指標: Stage-1 baseline / Agent-only / Final (override) ---
m_stage1 = compute_cls_metrics(eval_df.assign(pred=eval_df["stage1_pred"]), label_col="label", pred_col="pred", beta=2.0)
print("\n[Stage-1 baseline on eval_df]")
print(m_stage1)

covered_df = eval_df[(eval_df["agent_covered"] == 1) & (eval_df.get("error").isna() if "error" in eval_df.columns else True)].copy()
if len(covered_df) > 0:
    m_agent = compute_cls_metrics(covered_df.assign(pred=covered_df["agent_pred"]), label_col="label", pred_col="pred", beta=2.0)
    print("\n[Agent-only on covered subset]")
    print("  covered_rows:", len(covered_df))
    print(m_agent)
else:
    print("\n[Agent-only] no covered rows")

m_final = compute_cls_metrics(eval_df.assign(pred=eval_df["final_pred"]), label_col="label", pred_col="pred", beta=2.0)
print("\n[Final (agent override w/ fallback) on eval_df]")
print(m_final)

# group別
if "eval_group" in eval_df.columns:
    print("\n[By eval_group]")
    for g, sub in eval_df.groupby("eval_group"):
        m = compute_cls_metrics(sub.assign(pred=sub["final_pred"]), label_col="label", pred_col="pred", beta=2.0)
        print(f"  - {g}: n={len(sub)}  {m}")


[INFO] eval_df rows: 3000
[INFO] agent_covered: 3000 / 3000


,domain,source,ml_probability,stage1_decision,y_true,label,eval_group,ai_is_phishing,ai_confidence,ai_risk_level,processing_time,error,stage1_pred,agent_pred,agent_covered,final_pred
0,scelgocasa.com,jpcert,0.176781,handoff_to_agent,1,1,stage2_handoff,False,0.75,low,4.102432,None,0,False,1,0
1,dickrobinson.com,trusted,0.125159,handoff_to_agent,0,0,stage2_handoff,False,0.85,low,4.193526,None,0,False,1,0
2,4qd.co.uk,trusted,0.128810,handoff_to_agent,0,0,stage2_handoff,False,0.55,medium,9.007978,None,0,False,1,0
3,amazon-xq.top,certificates,0.615849,handoff_to_agent,1,1,stage2_handoff,True,0.75,high,3.198911,None,1,True,1,1
4,iscarpino.com,jpcert,0.174722,handoff_to_agent,1,1,stage2_handoff,False,0.85,low,8.348804,None,0,False,1,0



[Stage-1 baseline on eval_df]
ConfusionResult(TP=534, FP=166, TN=1791, FN=509, precision=0.7628571428571429, recall=0.5119846596356663, f1=0.612736660929432, fbeta=0.5480295566502463, fpr=0.08482370975983648)

[Agent-only on covered subset]
  covered_rows: 3000
ConfusionResult(TP=573, FP=204, TN=1753, FN=470, precision=0.7374517374517374, recall=0.5493767976989453, f1=0.6296703296703297, fbeta=0.578904829258436, fpr=0.10424118548799183)

[Final (agent override w/ fallback) on eval_df]
ConfusionResult(TP=573, FP=204, TN=1753, FN=470, precision=0.7374517374517374, recall=0.5493767976989453, f1=0.6296703296703297, fbeta=0.578904829258436, fpr=0.10424118548799183)

[By eval_group]
  - stage2_handoff: n=3000  ConfusionResult(TP=573, FP=204, TN=1753, FN=470, precision=0.7374517374517374, recall=0.5493767976989453, f1=0.6296703296703297, fbeta=0.578904829258436, fpr=0.10424118548799183)


In [11]:
# ============================================
# Cell 8: End-to-End評価（ALL test）: Stage-1 → Stage-2(handoff) → Agent上書き（fallbackあり）
# ============================================
import pandas as pd
import numpy as np

# Stage-2 handoff domains（ALL test 上のマスク）
handoff_domains = set(handoff_all["domain"].astype(str))
full_eval = full_df.copy()
full_eval["domain"] = full_eval["domain"].astype(str)
full_eval["stage2_handoff"] = full_eval["domain"].isin(handoff_domains).astype(int)

# agent予測（今回回した分だけ存在する可能性あり）
agent_for_merge = eval_df[["domain","agent_pred"]].copy() if "eval_df" in globals() else pd.DataFrame(columns=["domain","agent_pred"])
agent_for_merge = agent_for_merge.drop_duplicates(subset=["domain"], keep="first")

merged = full_eval.merge(agent_for_merge, on="domain", how="left")

# final_pred: agentが取れたら上書き、無ければ stage1_pred
merged["final_pred"] = pd.to_numeric(merged["agent_pred"], errors="coerce").fillna(merged["stage1_pred"]).astype(int)

# coverage（Stage-2 handoffのうちagent_predが取れた割合）
handoff_mask = merged["stage2_handoff"] == 1
covered = int(merged.loc[handoff_mask, "agent_pred"].notna().sum())
total_handoff = int(handoff_mask.sum())
print("\n[COVERAGE]")
print(f"  stage2_handoff_total = {total_handoff}")
print(f"  agent_covered_in_handoff = {covered}  (rate={covered/total_handoff if total_handoff else 0:.3f})")

# --- 指標: ALL ---
m_stage1_all = compute_cls_metrics(merged.assign(pred=merged["stage1_pred"]), label_col="label", pred_col="pred", beta=2.0)
m_final_all = compute_cls_metrics(merged.assign(pred=merged["final_pred"]), label_col="label", pred_col="pred", beta=2.0)

print("\n[ALL test] Stage-1 vs Final")
print("  Stage-1:", m_stage1_all)
print("  Final  :", m_final_all)

# --- AUTO subset（Stage-2 handoff外） ---
auto_mask = merged["stage2_handoff"] == 0
m_stage1_auto = compute_cls_metrics(merged.loc[auto_mask].assign(pred=merged.loc[auto_mask, "stage1_pred"]), label_col="label", pred_col="pred", beta=2.0)
m_final_auto = compute_cls_metrics(merged.loc[auto_mask].assign(pred=merged.loc[auto_mask, "final_pred"]), label_col="label", pred_col="pred", beta=2.0)

print("\n[AUTO subset] (not handed off)")
print("  n_auto:", int(auto_mask.sum()))
print("  Stage-1:", m_stage1_auto)
print("  Final  :", m_final_auto)

# --- HANDOFF subset（Stage-2 handoff内） ---
m_stage1_h = compute_cls_metrics(merged.loc[handoff_mask].assign(pred=merged.loc[handoff_mask, "stage1_pred"]), label_col="label", pred_col="pred", beta=2.0)
m_final_h = compute_cls_metrics(merged.loc[handoff_mask].assign(pred=merged.loc[handoff_mask, "final_pred"]), label_col="label", pred_col="pred", beta=2.0)

print("\n[HANDOFF subset] (Stage-2 handoff)")
print("  n_handoff:", int(handoff_mask.sum()))
print("  Stage-1:", m_stage1_h)
print("  Final  :", m_final_h)

# agent-only on covered subset inside handoff
covered_mask = handoff_mask & merged["agent_pred"].notna()
if covered_mask.any():
    m_agent_h = compute_cls_metrics(merged.loc[covered_mask].assign(pred=merged.loc[covered_mask, "agent_pred"]), label_col="label", pred_col="pred", beta=2.0)
    print("\n[Agent-only within HANDOFF covered subset]")
    print("  n_covered:", int(covered_mask.sum()))
    print("  Agent:", m_agent_h)
else:
    print("\n[Agent-only within HANDOFF] no covered rows")

# --- cost（任意） ---
cost_stage1 = cost_from_confusion(m_stage1_all, fn_cost=FN_COST, fp_cost=FP_COST, handoff_cost=HANDOFF_COST, handoff_count=0)
cost_final = cost_from_confusion(m_final_all, fn_cost=FN_COST, fp_cost=FP_COST, handoff_cost=HANDOFF_COST, handoff_count=total_handoff)

print("\n[COST] cost = FN_COST*FN + FP_COST*FP + HANDOFF_COST*handoff_count")
print("  Stage-1 only:", cost_stage1)
print("  Final (with handoff):", cost_final)

# 参考: gate metrics は Cell3 ですでに算出済みだが、見やすいよう再掲
gate_all = compute_gate_metrics((merged["stage1_pred"] != merged["label"]).astype(int), merged["stage2_handoff"])
print("\n[Gate metrics (reprint)]")
print(gate_all)



[COVERAGE]
  stage2_handoff_total = 7710
  agent_covered_in_handoff = 3000  (rate=0.389)

[ALL test] Stage-1 vs Final
  Stage-1: ConfusionResult(TP=61811, FP=538, TN=63496, FN=2222, precision=0.9913711527049351, recall=0.9652991426295816, f1=0.978161447041509, fbeta=0.9704032579651533, fpr=0.008401786550894836)
  Final  : ConfusionResult(TP=61850, FP=576, TN=63458, FN=2183, precision=0.9907730753211803, recall=0.9659082035825277, f1=0.9781826520848654, fbeta=0.9707808311202356, fpr=0.008995221288690384)

[AUTO subset] (not handed off)
  n_auto: 120357
  Stage-1: ConfusionResult(TP=60399, FP=104, TN=58915, FN=939, precision=0.9982810769713899, recall=0.984691382177443, f1=0.9914396631675709, fbeta=0.9873796406794068, fpr=0.001762144394178146)
  Final  : ConfusionResult(TP=60399, FP=104, TN=58915, FN=939, precision=0.9982810769713899, recall=0.984691382177443, f1=0.9914396631675709, fbeta=0.9873796406794068, fpr=0.001762144394178146)

[HANDOFF subset] (Stage-2 handoff)
  n_handoff: 7710

In [12]:
# ============================================
# Cell 9: 結果保存（共有用）
# ============================================
from datetime import datetime
from pathlib import Path
import json

OUT_DIR = Path(RESULTS_DIR) / "stage2_validation"
OUT_DIR.mkdir(parents=True, exist_ok=True)

ts = datetime.now().strftime("%Y-%m-%d_%H%M%S")

# eval_df（今回agentを回した集合）
eval_path = OUT_DIR / f"eval_df__n{len(eval_df)}__ts_{ts}.csv"
eval_df.to_csv(eval_path, index=False)

# merged（ALL test end-to-end）
merged_path = OUT_DIR / f"all_test_merged__ts_{ts}.csv"
merged.to_csv(merged_path, index=False)

# summary json
summary = {
    "run_id": RUN_ID,
    "n_all_test": int(len(full_df)),
    "n_stage2_handoff": int(full_df["stage2_handoff"].sum()),
    "agent_covered_in_handoff": int((merged["stage2_handoff"].eq(1) & merged["agent_pred"].notna()).sum()),
    "metrics_stage1_all": m_stage1_all.__dict__,
    "metrics_final_all": m_final_all.__dict__,
    "metrics_stage1_auto": m_stage1_auto.__dict__,
    "metrics_final_auto": m_final_auto.__dict__,
    "metrics_stage1_handoff": m_stage1_h.__dict__,
    "metrics_final_handoff": m_final_h.__dict__,
    "gate_all": gate_all,
    "cost": {
        "FN_COST": FN_COST,
        "FP_COST": FP_COST,
        "HANDOFF_COST": HANDOFF_COST,
        "stage1_only": cost_stage1,
        "final": cost_final,
    },
    "params": {
        "N_SAMPLE": os.environ.get("N_SAMPLE"),
        "N_BENIGN_SAMPLE": os.environ.get("N_BENIGN_SAMPLE"),
        "N_BENIGN_HARD_SAMPLE": os.environ.get("N_BENIGN_HARD_SAMPLE"),
        "RANDOM_STATE": os.environ.get("RANDOM_STATE"),
    },
}

sum_path = OUT_DIR / f"summary__ts_{ts}.json"
sum_path.write_text(json.dumps(summary, ensure_ascii=False, indent=2), encoding="utf-8")

print("[SAVED]", eval_path)
print("[SAVED]", merged_path)
print("[SAVED]", sum_path)


[SAVED] artifacts/2026-01-03_222059/results/stage2_validation/eval_df__n3000__ts_2026-01-04_070814.csv
[SAVED] artifacts/2026-01-03_222059/results/stage2_validation/all_test_merged__ts_2026-01-04_070814.csv
[SAVED] artifacts/2026-01-03_222059/results/stage2_validation/summary__ts_2026-01-04_070814.json
